In [2]:
import json
import pickle
import numpy as np
import pandas as pd
from utils import *
pd.set_option('display.max_rows',500)
pd.set_option('display.max_columns',500)
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.colors import ListedColormap
import matplotlib.colors as mcolors
%matplotlib inline

In [ ]:
model_name_list = [250,500,1000,2000,3000,4000,5000,6000]

for voll_c in model_name_list:
    print(voll_c)
    model_name = "modified_td_coordination_" + str(voll_c)
    main_path = "/Users/ashutoshshukla/Downloads/" + model_name + "/"

    with open(main_path + "model_params.json", 'r') as f:
        params = json.load(f)   
    params["path_to_input"] = os.getcwd() + "/data/192_Scenario/"

    with open(main_path + "model_scenarios.json", 'r') as f:
        model_scenarios_string = json.load(f)

    model_scenarios = {}
    for k in model_scenarios_string:
        model_scenarios[int(k)] = model_scenarios_string[k]

    base_model = three_stage_model(params, model_scenarios)
    base_model.model.update()
    sol_path = main_path + "solution.sol"
    base_model.model.read(sol_path)
    base_model.model.update()

    hardening_decisions = {}
    tiger_dam_decisions = {}

    for sub_id in df_flood.index:
        hardening_decisions[sub_id] = int(base_model.x_mit[sub_id].Start*params["mit_level"])
    for sub_id in df_flood.index:
        for j in model_scenarios:
            tiger_dam_decisions[str(sub_id) + "_" + str(j)] = int(base_model.x_prep[sub_id,j].Start*params["prep_level"])


    df_plot = pd.DataFrame(index=df_flood.index, columns=np.arange(0,54,1))
    lister = []
    for i in df_plot.index:
        for j in df_plot.columns:
            temp = str(i) + "_" + str(j)
            td_value = tiger_dam_decisions[temp]
            flood = df_flood.loc[i, model_scenarios[j]].max()
            if td_value == 0:
                if flood == 0:
                    df_plot.loc[i,j] = 0.0
                else:
                    df_plot.loc[i,j] = 1.0
            else:
                if td_value > flood:
                    df_plot.loc[i,j] = 2.0
                elif td_value == flood:
                    df_plot.loc[i,j] = 3.0
                else:
                    df_plot.loc[i,j] = 4.0
        if hardening_decisions[i] > 0:
            lister.append(5.0)
        else:
            lister.append(6.0)

    df_plot["hardening"] = lister
    df_plot["hardening1"] = lister
    df_plot["hardening2"] = lister
    df_plot["hardening3"] = lister

    colormap_mapping = {
        0.0: 'red',
        1.0: 'green',
        2.0: 'blue',
        3.0: 'purple',
        4.0: 'orange',
        5.0: "black",
        6.0: "yellow"
    }
    colors = [colormap_mapping[i] for i in range(7)]
    cmap = mpl.colors.ListedColormap(colors)

    plt.figure(figsize=(10,5))
    plt.matshow(df_plot.iloc[:,:].values.astype(float),cmap=cmap)
    cb = plt.colorbar(shrink=0.8)
    labels = np.arange(0,7, 0.2)
    loc    = [0.45,1.3,2.1,3,3.8,4.7,5.6]
    cb.set_ticks(loc)
    l1 = ["td = max_flood = 0", "td = 0 < max_flood", "td > max_flood > 0", "td = max_flood > 0",
          "0 <td < max_flood", "hardened", "not hardened"]

    cb.set_ticklabels(l1)
    plt.tick_params(which = 'both', size = 0, labelsize = 0)
    plt.tight_layout()
    plt.xlabel("Preparedness Models")
    plt.ylabel("Substations")
    plt.savefig("output_plots/" + model_name + ".pdf", format="pdf", bbox_inches="tight")
    #plt.show()